<a href="https://colab.research.google.com/github/fantaxiah/DeepLearningSpring23/blob/main/DLProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import module
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import xlrd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import shutil
import tensorflow as tf 

from sklearn.decomposition import LatentDirichletAllocation
from sklearn import preprocessing
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import FastICA

# import squarify

from keras.models import Model #import model  
from keras.layers import Dense, Input

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D,MaxPooling2D,Flatten,AveragePooling2D,Dropout,BatchNormalization,Activation
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
from keras.models import load_model


In [ ]:
# Load data from xlsx file
def Load_Data(fname):
    ExcelFile = xlrd.open_workbook(fname)

    # Read training data
    Sheet_1 = ExcelFile.sheet_by_index(0)
    Mtraindatalist = list()
    for i in range(Sheet_1.ncols-1):
        tpcol = Sheet_1.col_values(i+1, 1)
        Mtraindatalist.append(tpcol)

    Mtraindata = np.array(Mtraindatalist)

    # Read training label
    Sheet_1 = ExcelFile.sheet_by_index(1)
    Vtrainlabel = np.array(Sheet_1.col_values(0,1))

    # Read validation data
    Sheet_1 = ExcelFile.sheet_by_index(2)
    Mvaldatalist = list()
    for i in range(Sheet_1.ncols-1):
        tpcol = Sheet_1.col_values(i+1, 1)
        Mvaldatalist.append(tpcol)

    Mvaldata = np.array(Mvaldatalist)

    # Read val label
    Sheet_1 = ExcelFile.sheet_by_index(3)
    Vvallabel = np.array(Sheet_1.col_values(0,1))

    Mtraindata = Mtraindata.T
    Mvaldata = Mvaldata.T
    
    # Return matrix is genes*samples
    return Mtraindata, Vtrainlabel, Mvaldata, Vvallabel

In [ ]:
# Oversampling by SMOTE
# DataSet should be genes*samples
def Over_Sampling(DataSet, DataLabel):
    X_res, y_res = SMOTE(random_state=3).fit_resample(DataSet.T, DataLabel)
    
    # Return matrix is genes*samples
    return X_res.T, y_res

In [ ]:
# Min_Max normalization
# DataSet should be genes*samples
def MinMax_Norm(DataSet):
    min_max_scaler = preprocessing.MinMaxScaler()
    min_max_scaler.fit(DataSet.T)

    DataSet = min_max_scaler.transform(DataSet.T)
    
    # Return matrix is genes*samples
    return DataSet.T

In [ ]:
# Build Topic Model
# geneArray should be genes*samples
def Topic_Model(geneArray, n_start = 30, n_end = 150, interval = 4, n_iter = 30):

    trans_geneArray = geneArray.T
    
    n_topics = range(n_start, n_end, interval)
    perplexityLst = [1.0]*len(n_topics)
      
    lda_models = []

    for idx, n_topic in enumerate(n_topics):
        print ("Start topic searching...: %d in %d"%(n_topic, n_end))
        lda = LatentDirichletAllocation(n_components=n_topic,
                                    max_iter=n_iter,
                                    learning_method='batch',
                                    n_jobs=-1,
                                    random_state=100,
#                                    evaluate_every=200,
                                    perp_tol=0.1, #default                                       
                                    doc_topic_prior=0.4, #default
                                    topic_word_prior=0.7, #default
                                    verbose=1)
        lda.fit(trans_geneArray)
        perplexityLst[idx] = lda.perplexity(trans_geneArray)
        lda_models.append(lda)
        
    best_index = perplexityLst.index(min(perplexityLst))
    best_n_topic = n_topics[best_index]
    best_model = lda_models[best_index]
    print ("The best topic number: %d"%(best_n_topic))
    
    # return matrix is nTopics x Genes
    return best_model.components_, perplexityLst

In [ ]:
# Build FastICA Model
# geneArray should be genes*samples
def FastICA_Model(geneArray, nComp = 32):

    np.random.seed(100)  # for reproducibility
    
    # Compute ICA  (reconstruct:  decomp_components * decomp_mixarray.T)
    ica = FastICA(n_components=nComp, whiten = True, random_state = 100)
    decomp_components = ica.fit_transform(geneArray)  # Calculate components_: Genes x nComps
    # decomp_mixarray = ica.mixing_  # Get estimated mixing matrix: samples x % of nComps

    final_weights = np.abs(decomp_components)   # Genes x nComps
    
    # Competion Model
    n_genes = final_weights.shape[0]
    n_comps = final_weights.shape[1]

    for i in range(n_genes):
        max_weight = np.max(final_weights[i,:])
    
        for j in range(n_comps):
            if final_weights[i,j] != max_weight:
                final_weights[i,j] = 0
        #end_for_j  
    #end_for_i
    
    # return matrix is nComps x Genes
    return final_weights.T

In [ ]:
# Build AutoEncoder Model
# geneArray should be genes*samples    
def Auto_Encoder(geneArray, encoding_dim = 32, epch=50, batchsize = 128):
    
    np.random.seed(100)  # for reproducibility
    x_train = geneArray.T
    
    len_feature = x_train.shape[1]
    # this is our input placeholder  
    input_img = Input(shape=(len_feature,))

    # coding layers  
    encoded = Dense(1280, activation='relu', name='encode_layer1')(input_img)
    encoded = Dense(320, activation='relu', name='encode_layer2')(encoded)
    encoded = Dense(128, activation='relu', name='encode_layer3')(encoded)
    encoded = Dense(64, activation='relu', name='encode_layer4')(encoded)
    encoder_output = Dense(encoding_dim)(encoded)

    # decoding layers  
    decoded = Dense(64, activation='relu', name='decode_layer1')(encoder_output)
    decoded = Dense(128, activation='relu', name='decode_layer2')(decoded)
    decoded = Dense(320, activation='relu', name='decode_layer3')(decoded)
    decoded = Dense(1280, activation='relu', name='decode_layer4')(decoded)
    decoded = Dense(len_feature, activation='tanh')(decoded)

    # autoencoder model 
    autoencoder = Model(inputs=input_img, outputs=decoded)
  
    # encoder model
    # encoder = Model(inputs=input_img, outputs=encoder_output)

    # compile autoencoder  
    autoencoder.compile(optimizer='adam', loss='mse')

    # training  
    autoencoder.fit(x_train, x_train, epochs=epch, batch_size=batchsize, shuffle=True)  

    autoencoder.save_weights('coder_weights.h5')

    # obtained the weights
    l1 = autoencoder.get_layer('encode_layer1').get_weights()
    l2 = autoencoder.get_layer('encode_layer2').get_weights()
    l3 = autoencoder.get_layer('encode_layer3').get_weights()
    l4 = autoencoder.get_layer('encode_layer4').get_weights()

    # Output weights
    a1 = l1[0]
    a2 = l2[0]
    a3 = l3[0]
    a4 = l4[0]

    final_weights = np.dot(np.dot(np.dot(a1, a2), a3), a4) # Genes x n_pcs
    final_weights = final_weights - np.min(final_weights) # Genes x n_pcs
    
    n_genes = final_weights.shape[0]
    n_comps = final_weights.shape[1]

    for i in range(n_genes):
        max_weight = np.max(final_weights[i,:])
    
        for j in range(n_comps):
            if final_weights[i,j] != max_weight:
                final_weights[i,j] = 0
        #end_for_j
  
    #end_for_i
    
    # return matrix is nComps x Genes
    return final_weights.T 

In [ ]:
# use pretrained weights for coding
# the newwork structure should be same as that in Auto_Encode function
def Auto_Encoder_Pretrained(geneArray, filename_pretrained_weights, encoding_dim = 32):    
    
    x_train = geneArray.T
    
    len_feature = x_train.shape[1]
    # this is our input placeholder  
    input_img = Input(shape=(len_feature,))

    # coding layers  
    encoded = Dense(1280, activation='relu', name='encode_layer1')(input_img)
    encoded = Dense(320, activation='relu', name='encode_layer2')(encoded)
    encoded = Dense(128, activation='relu', name='encode_layer3')(encoded)
    encoded = Dense(64, activation='relu', name='encode_layer4')(encoded)
    encoder_output = Dense(encoding_dim)(encoded)

    # decoding layers  
    decoded = Dense(64, activation='relu', name='decode_layer1')(encoder_output)
    decoded = Dense(128, activation='relu', name='decode_layer2')(decoded)
    decoded = Dense(320, activation='relu', name='decode_layer3')(decoded)
    decoded = Dense(1280, activation='relu', name='decode_layer4')(decoded)
    decoded = Dense(len_feature, activation='tanh')(decoded)

    # autoencoder model 
    autoencoder = Model(inputs=input_img, outputs=decoded)
  
    # encoder model
    # encoder = Model(inputs=input_img, outputs=encoder_output)

    # compile autoencoder  
    autoencoder.compile(optimizer='adam', loss='mse')
    
    autoencoder.load_weights(filename_pretrained_weights)
    
    # obtained the weights
    l1 = autoencoder.get_layer('encode_layer1').get_weights()
    l2 = autoencoder.get_layer('encode_layer2').get_weights()
    l3 = autoencoder.get_layer('encode_layer3').get_weights()
    l4 = autoencoder.get_layer('encode_layer4').get_weights()

    # Output weights
    a1 = l1[0]
    a2 = l2[0]
    a3 = l3[0]
    a4 = l4[0]

    final_weights = np.dot(np.dot(np.dot(a1, a2), a3), a4) # Genes x n_pcs
    final_weights = final_weights - np.min(final_weights) # Genes x n_pcs
    
    n_genes = final_weights.shape[0]
    n_comps = final_weights.shape[1]

    for i in range(n_genes):
        max_weight = np.max(final_weights[i,:])
    
        for j in range(n_comps):
            if final_weights[i,j] != max_weight:
                final_weights[i,j] = 0
        #end_for_j
  
    #end_for_i
    
    # return matrix is nComps x Genes
    return final_weights.T 

In [ ]:
# Compile the results from Topic model
# best_model_float should be float matrix with ntopics*genes
def Topicmodel_Results_Compile(best_model_float, nodecut = 500):
    
    shapeTopics = best_model_float.shape
    
    indexarray = np.argsort(-best_model_float,axis=1)
    
    regroup_indexarray = []  # list
    first_layer = [nodecut]*shapeTopics[0]     # list
    
    for i in range(shapeTopics[0]):
        tpidx = indexarray[i]
        regroup_indexarray.append(tpidx[0:nodecut])
    #end_for_i
    
    #regroup_indexarray(list); first_layer(list) 
    return regroup_indexarray, first_layer   

In [ ]:
# Compile the results from FastICA and AutoEncoder
# best_model_float should be float matrix with ntopics*genes
def Results_Compile(best_model_float, th = 1):
    
    shapeTopics = best_model_float.shape
    
    indexarray = np.argsort(-best_model_float,axis=1)
    regroup_indexarray = []  # list
    first_layer = []     # list
    
    for i in range(shapeTopics[0]):
        if np.std(best_model_float[i]) !=0:
            tpidx = indexarray[i]
            tp = best_model_float[i, tpidx]
            t_total = sum(tp)
            
            for j in range(1, shapeTopics[1]):
                tp_acum = sum(tp[0:j])/t_total
                if tp_acum >= th:
                    regroup_indexarray.append(tpidx[0:j])
                    first_layer.append(j)
                    break
                #end_if
            #end_for_j
        #end_if
    #end_for_i
    
    #regroup_indexarray(list); first_layer(list) 
    return regroup_indexarray, first_layer 

In [ ]:
# log2 transformed gene exp array
# input array should be genes*samples
def Log_transform(exp_matrix):
    
    n_samples = exp_matrix.shape[1]
    n_gene = exp_matrix.shape[0]
    
    for i in range(n_samples):
        if i ==0:
            tp = exp_matrix[:,i]
            log_tran_matrix = np.log2(tp)
            log_tran_matrix = log_tran_matrix.reshape(n_gene, 1)
        else:
            tp = exp_matrix[:,i]
            tp = np.log2(tp)
            tp = tp.reshape(n_gene,1)
            log_tran_matrix = np.concatenate((log_tran_matrix, tp), axis = 1)
    
    return log_tran_matrix

In [ ]:
# Construct TreeMap for a sample
# exp_vector is a gene expression vector for a sample
# top_layer_list, gene_im_list can be obtained from ResultsCompile_* functions
def Draw_Treemap(exp_vector, top_layer_list, gene_imp_list, im_name = '_tmp_fig.png', high_inch = 5, width_inch = 5, set_dpi = 200):
    
    #exp_vector = np.log2(exp_vector)
    
    exp_gene_list = []
    for i in range(len(top_layer_list)):
        tp_exp = exp_vector[gene_imp_list[i]]
        exp_gene_list.append(tp_exp)
    #end_for_i
    
    x = 0
    y = 0
    width = 1
    height = 1
    
    mini = np.min(exp_vector)
    maxi = np.max(exp_vector)
    #mini = 0.001
    #maxi = 30000

    cmap = matplotlib.cm.jet
    norm = matplotlib.colors.Normalize(vmin=mini, vmax=maxi)

    tp_array = np.array(top_layer_list)
    ori_topic_idx = np.argsort(-tp_array)
    
    top_layer_list.sort(reverse=True)
    first_values = squarify.normalize_sizes(top_layer_list, width, height)
    
    # Start to prepare figure
    fig = plt.figure(frameon = False)
    fig.set_size_inches(width_inch, high_inch)

    rects = squarify.squarify(first_values, x, y, width, height)
    
    for i in range(len(top_layer_list)):
        tp_ax = plt.Axes(fig, [rects[i]['x'], rects[i]['y'], rects[i]['dx'], rects[i]['dy']], )
        tplen = top_layer_list[i]
        tplist = [1]*tplen
        tp_values = squarify.normalize_sizes(tplist, rects[i]['dx'], rects[i]['dy'])
        
        mycolor = [cmap(norm(value)) for value in exp_gene_list[ori_topic_idx[i]]]
        tp_ax = squarify.plot(tp_values, color=mycolor, ax=tp_ax)
        tp_ax.set_axis_off()
        fig.add_axes(tp_ax)
    
    #end_for_i
    plt.savefig(im_name, dpi = set_dpi)
    plt.cla()
    plt.close('all')
    
    return 0

In [ ]:
# Generate the images for training and validation
# dataset should be genes x samples
def Batch_ImageOrganizor(dataset, datalabel, top_layer_list, gene_imp_list, path = './data/', dirname = 'Dataset'):
    
    n_class = np.unique(datalabel)
    curr_path = os.getcwd()
    generate_path = path+dirname
    
    if os.path.exists(generate_path) == False:
        os.makedirs(generate_path)
        
        for i in range(len(n_class)):
            os.chdir(curr_path)
            Classname = 'C'+np.str(i+1).zfill(3)
            os.makedirs(generate_path+'/'+Classname)
            os.chdir(generate_path+'/'+Classname)
            
            idx_samples = np.argwhere(datalabel == i)
            num_samples = idx_samples.shape[0]
            idx_samples = idx_samples.reshape(num_samples)

            exp_data_inclass = dataset[:,idx_samples]
                    
            for j in range(num_samples):
                tp_exp = exp_data_inclass[:,j]
                im_file_name = Classname+'_P'+str(j+1).zfill(5)+'.jpg'
                
                #ex
                if j==0:
                    tp=Draw_Treemap(tp_exp, top_layer_list, gene_imp_list, im_name = im_file_name)

                tp = Draw_Treemap(tp_exp, top_layer_list, gene_imp_list, im_name = im_file_name)
            #end_for_j
        #end_for_i
    else:
        print('The path: <'+generate_path+'> exists!')
    
    os.chdir(curr_path)
    return 0

In [ ]:
# randomly Split samples from the whole-training set into sub_train and sub_test
# According to the split_rate    
def Sample_Split(fromdir = './data/Train', split_rate = 0.3):
    
    if (os.path.exists('./split_data')):
        shutil.rmtree('./split_data')
    
    todir_train = './split_data/Train'
    todir_test = './split_data/Test'
    
    all_class = os.listdir(fromdir)
    
    np.random.seed(100)
    
    for i in range(len(all_class)):
        
        curr_path = fromdir+'/'+all_class[i]
        all_files = os.listdir(curr_path)
        
        to_path_train = todir_train+'/'+all_class[i]
        to_path_test = todir_test+'/'+all_class[i]
        os.makedirs(to_path_train)
        os.makedirs(to_path_test)
        
        topn = round(len(all_files)*split_rate)
        idx = np.arange(0,len(all_files))
        idx = np.random.permutation(idx)
        
        for j in range(len(idx)):
            
            if (j<=topn):
                from_file = curr_path+'/'+all_files[idx[j]]
                to_file = to_path_test+'/'+all_files[idx[j]]
                shutil.copyfile(from_file, to_file)
            else:
                from_file = curr_path+'/'+all_files[idx[j]]
                to_file = to_path_train+'/'+all_files[idx[j]]
                shutil.copyfile(from_file, to_file)
            #end_if
        #end_for_j

    #end_for_i
    
    return 0

In [ ]:
# Define a CNN model
def MiniModel(input_shape, n_class):
    images = Input(input_shape)
    net = Unit(images,32)
    net = MaxPooling2D(pool_size=(2,2))(net)
    net = Unit(net,64)
    net = MaxPooling2D(pool_size=(2,2))(net)
    net = Unit(net,128)
    net = MaxPooling2D(pool_size=(2,2))(net)
    #net = Unit(net,256)
    #net = MaxPooling2D(pool_size=(2,2))(net)
    #net = Unit(net,512)
    #net = AveragePooling2D(pool_size=(2,2))(net)
    net = Flatten()(net)
    #net = BatchNormalization()(net)
    net = Dense(units=256,activation="selu")(net)
    net = Dropout(0.5)(net)
    net = Dense(units=n_class,activation="softmax")(net)
    model = Model(inputs=images,outputs=net)
    return model

    def CNN_Construction(train_dir = './data/Train', val_dir = './data/Validation', target_w = 100, target_h =100, batchsize = 32, ranseed = 100, n_epochs = 100):
    
    train_datagen = ImageDataGenerator(rescale = 1/255.,
                                       rotation_range=0,
                                       width_shift_range=0,
                                       height_shift_range=0,
                                       shear_range=0,
                                       zoom_range=0,
                                       fill_mode='nearest')
    val_datagen = ImageDataGenerator(rescale = 1/255.)
    
    train_generator = train_datagen.flow_from_directory(
                                        train_dir,
                                        target_size=(target_w, target_h),
                                        batch_size=batchsize,
                                        seed = ranseed,
                                        color_mode='rgb',
                                        class_mode='categorical')
    val_generator = val_datagen.flow_from_directory(
                                        val_dir,
                                        target_size=(target_w, target_h),
                                        batch_size=batchsize,
                                        seed = ranseed,
                                        color_mode='rgb',
                                        class_mode='categorical')
    
    n_class = train_generator.num_classes
    n_train = train_generator.n
    n_val = val_generator.n
    
    input_shape = (target_h, target_w, 3)
    
    model = MiniModel(input_shape, n_class)
    sgd = SGD(lr=0.0001, momentum=0.9, decay=0.001/n_epochs, nesterov=False)
    #sgd = SGD(learning_rate=0.0001)
    model.compile(optimizer=sgd,loss="categorical_crossentropy",metrics=["accuracy"])
    
    
    best_AUC = 0
    best_ACC = 0
    
    best_model_filepath = './saved_best_model.hdf5'
    
    #new start
    checkpointer = ModelCheckpoint(filepath=best_model_filepath, monitor='val_accuracy', verbose=2, save_best_only=True, mode='max')
    
    fitted_model = model.fit(
                                train_generator,
                                steps_per_epoch=n_train//batchsize,
                                epochs=n_epochs,
                                validation_data=val_generator,
                                validation_steps=n_val//batchsize,
                                verbose=1, 
                                callbacks=[checkpointer])
       
    # Test the model in training sets
    model = load_model(best_model_filepath)
    
    ld_time = int(np.ceil(n_val/batchsize))
    
    for i in range(ld_time):
        if (i == 0):
            data, val_true_label = val_generator.next()
        else:
            data, tplabel = val_generator.next()
            val_true_label = np.concatenate((val_true_label, tplabel), axis = 0)
    #end_for_i
    val_true_label = val_true_label.argmax(1)
    
    val_predict = model.predict(val_generator)
    val_predict = val_predict.argmax(1)
    best_AUC = metrics.roc_auc_score(val_true_label, val_predict)
    best_ACC = metrics.accuracy_score(val_true_label, val_predict)
    
    return best_AUC, best_ACC

IndentationError: ignored

In [ ]:
# Using trained CNN weights for predicion    
def CNN_Prediction(model_file = './saved_best_model.hdf5', val_dir = './data/Validation', target_w = 100, target_h =100, batchsize = 32, ranseed = 100, n_epochs = 100):
    
    val_datagen = ImageDataGenerator(rescale = 1/255.)
    
    val_generator = val_datagen.flow_from_directory(
                                        val_dir,
                                        target_size=(target_w, target_h),
                                        batch_size=batchsize,
                                        seed = ranseed,
                                        color_mode='rgb',
                                        class_mode='categorical')
    
    n_val = val_generator.n
    model = load_model(model_file)
    
    ld_time = int(np.ceil(n_val/batchsize))
    
    for i in range(ld_time):
        if (i == 0):
            data, val_true_label = val_generator.next()
        else:
            data, tplabel = val_generator.next()
            val_true_label = np.concatenate((val_true_label, tplabel), axis = 0)
    #end_for_i
    val_true_label = val_true_label.argmax(1)
    
    val_predict = model.predict(val_generator)
    val_predict = val_predict.argmax(1)
    
    # output
    print("ACC Score (Validation):  %5.4f"%metrics.accuracy_score(val_true_label, val_predict))
    print ("AUC Score (Validation):  %5.4f"%metrics.roc_auc_score(val_true_label,val_predict))
    pred_mcc = metrics.matthews_corrcoef(val_true_label, val_predict)
    print ("MCC Score (Validation):  %5.4f"%pred_mcc)

    tn, fp, fn, tp = metrics.confusion_matrix(val_true_label, val_predict, labels=[0, 1]).ravel()
    print("Validation: (TP, FP, TN, FN):  %d, %d, %d, %d"%(tp, fp, tn, fn))
    
    pred_acc = metrics.accuracy_score(val_true_label, val_predict)
    pred_AUC = metrics.roc_auc_score(val_true_label, val_predict)
    return pred_acc, pred_AUC, pred_mcc

if __name__ == '__main__':
  
    # constants
    looptime = 1
    methods = 2         # 0 - Autoencoder; 1 - FastICA; 2 - Topic Model
    datafile = './Dataset_SEQC_NB_RPM_GSE62564_EFS.xlsx'
    
    if (methods == 0):
        #best_model_float = Auto_Encoder(all_data)
        #regroup_indexarray, first_layer = Results_Compile(best_model_float)
        custom_path = './imgdata_Autoencoder/'
    elif (methods == 1):
        #best_model_float = FastICA_Model(all_data)
        #regroup_indexarray, first_layer = Results_Compile(best_model_float)
        custom_path = './imgdata_FastICA/'
    else:
        #all_data = all_data*100
        #all_data = all_data.astype(np.int)
        #best_model_float, perlist = Topic_Model(all_data)
        #savedata = best_model_float.T
        #tp = np.savetxt('_temp_best_model_matrix.csv', savedata, delimiter=',')
        #ps = np.array(perlist)
        #ps = ps.T
        #tp = np.savetxt('_temp_all_perplexity.csv', ps, delimiter=',')
        #regroup_indexarray, first_layer = Topicmodel_Results_Compile(best_model_float, nodecut = 600)
        custom_path = './imgdata_TopicModel/'
    
    res = [['train_acc', 'train_auc', 'val_acc', 'val_auc', 'val_mcc']]
    for i in range(looptime):
        frompath = custom_path+'Train'    
        tp = Sample_Split(fromdir = frompath, split_rate = 0.1)
    
        validationdir = custom_path+'Validation'
        
        best_AUC, best_ACC = CNN_Construction(train_dir = './split_data/Train', val_dir = './split_data/Test',target_w = 256, target_h =256, batchsize = 20, n_epochs = 100)
        print("AUC Score (Training):  %5.4f"%best_AUC)
        print("ACC Score (Training):  %5.4f"%best_ACC)
        pred_acc, pred_AUC, pred_mcc = CNN_Prediction(model_file = './saved_best_model.hdf5', val_dir = validationdir, target_w = 256, target_h =256, batchsize = 20, n_epochs = 100)
    
        tpres = [best_ACC, best_AUC, pred_acc, pred_AUC, pred_mcc]
        res.append(tpres)
        
    with open("Loop_results.txt","w") as f:
        for i in res:
            i = str(i).strip('[').strip(']').replace('\'','')+'\n'
            f.writelines(i)

FileNotFoundError: ignored